In [1]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [54]:
# gov_hospitals = pd.read_csv('Gov_Hospitals.csv')
mail_match = pd.read_csv('Matched_On_Mailing_Address.csv')
# loc_match = pd.read_csv('Matched_On_Practice_Location.csv')

In [18]:
npi = pd.read_csv('NPI_with_Taxonomy.csv')

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (9,12,17,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [20]:
npi[['NPI','Prov_Other_Org_Nm']]

,NPI,Prov_Other_Org_Nm
0,1003000118,NaN
1,1003000209,NaN
2,1003000217,NaN
3,1003000233,NaN
4,1003000258,LODI PHARMACY
...,...,...
1327685,1992999767,NaN
1327686,1992999775,NaN
1327687,1992999809,NaN
1327688,1992999841,MIDTOWN DENTAL CENTER


In [49]:
loc_match = loc_match.drop(columns = ['NAME_FLAG','ADDRESS_FLAG'])
loc_match = pd.merge(loc_match, npi[['NPI','Prov_Other_Org_Nm']], on='NPI')
loc_match = pd.merge(loc_match, gov_hospitals[['NAME','ADDRESS','ID']], on=['NAME','ADDRESS'])

In [50]:
loc_match = loc_match.rename(columns={'Prov_Other_Org_Nm':'OTHER_NAME'})

In [55]:
loc_match = loc_match.rename(columns={'NPI_ADDRESS':'NPI_ADDRESS_LOC'})
mail_match = mail_match.rename(columns={'NPI_ADDRESS':'NPI_ADDRESS_MAIL'})

In [52]:
loc_match = pd.merge(loc_match, npi[['NPI','Prov_1st_Line_Bus_Mail_Addr']], on='NPI').rename(columns={'Prov_1st_Line_Bus_Mail_Addr':'NPI_ADDRESS_MAIL'})

In [56]:
mail_match = pd.merge(mail_match, npi[['NPI','Prov_1st_Line_Bus_Prac_Loc_Addr']], on='NPI').rename(columns={'Prov_1st_Line_Bus_Prac_Loc_Addr':'NPI_ADDRESS_LOC'})

In [57]:
all_match = pd.concat([loc_match, mail_match])

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [60]:
all_match = all_match.fillna('None')

In [62]:
scores = []
for row in all_match.itertuples():
    mail_score = fuzz.ratio(row.NPI_ADDRESS_MAIL, row.ADDRESS)
    loc_score = fuzz.ratio(row.NPI_ADDRESS_LOC, row.ADDRESS)
    name_score = fuzz.ratio(row.NPI_NAME, row.NAME)
    other_score = fuzz.ratio(row.OTHER_NAME, row.NAME)
    
    add_score = max([mail_score,loc_score])
    name_score = max([name_score,other_score])
    total_score = add_score + name_score
    scores.append(total_score)
    if total_score < 150:
        print('PROBABLY WRONG')
        print(f'{row.NPI_NAME} matched to {row.NAME}')
        print('')

PROBABLY WRONG
CAPE FEAR NEONATOLOGY matched to CAPE FEAR VALLEY MEDICAL CENTER

PROBABLY WRONG
CHRIST HOSPITAL matched to DRAKE CENTER FOR POST-ACUTE CARE, LLC

PROBABLY WRONG
CHEROKEE INDIAN HOSPITAL AUTHORITY matched to BELAU NATIONAL HOSPITAL

PROBABLY WRONG
ENCOMPASS HEALTH REHABILITATION HOSPITAL OF TREASURE COAST, INC. matched to HEALTHSOUTH TREASURE COAST REHABILITATION

PROBABLY WRONG
OAKWOOD HOSPITAL matched to BEAUMONT HOSPITAL DEARBORN

PROBABLY WRONG
IASIS GLENWOOD REGIONAL MEDICAL CENTER LP matched to LOUISIANA EXTENDED CARE HOSPITAL OF WEST MONROE

PROBABLY WRONG
MERITER HOSPITAL, INC. matched to MERITER CHILD AND ADOLESCENT PSYCH HOSPITAL

PROBABLY WRONG
GEISINGER MEDICAL CENTER matched to SELECT SPECIALTY HOSPITAL DANVILLE

PROBABLY WRONG
NOLAND HOSPITAL ANNISTON II, LLC matched to NORTHEAST ALABAMA REGIONAL MEDICAL CENTER

PROBABLY WRONG
KITTSON DENTAL CLINIC PA matched to KITTSON MEMORIAL HOSPITAL

PROBABLY WRONG
ST CLOUD EMERGENCY PHYSICIANS LLC matched to ST CLOUD 

PROBABLY WRONG
TRINITY HOSPITALS matched to TRINITY HOSPITAL - SAINT JOSEPHS

PROBABLY WRONG
FRANCISCAN HEALTH OLYMPIA FIELDS & CHICAGO HEIGHTS matched to FRANCISCAN ALLIANCE, INC. - DBA FRANCISCAN ST. JAMES HEALTH-CHICAGO HEIGHTS

PROBABLY WRONG
OPTIM ORTHOPEDICS, LLC matched to OPTIM MEDICAL CENTER TATTNALL

PROBABLY WRONG
BOSTON VA HEALTH CARE NETWORK matched to VA BOSTON HEALTHCARE SYSTEM - JAMAICA PLAIN

PROBABLY WRONG
SSM REGIONAL HEALTH SERVICES matched to SSM HEALTH ST. FRANCIS HOSPITAL

PROBABLY WRONG
THE MOSES H CONE MEMORIAL HOSPITAL OPERATING CORPORATION matched to SELECT SPECIALTY HOSPITAL - GREENSBORO

PROBABLY WRONG
FARGO VAMC matched to FARGO VA MEDICAL CENTER

PROBABLY WRONG
NIOBRARA COUNTY HOSPITAL DISTRICT matched to NIOBRARA HEALTH AND LIFE CENTER

PROBABLY WRONG
WESLEY WOODS LONG TERM HOSPITAL, INC matched to SELECT SPECIALTY HOSPITAL - NORTHEAST ATLANTA

PROBABLY WRONG
SAUK PRAIRIE AMBULANCE ASSOCIATION matched to SAUK PRAIRIE MEM HSPTL

PROBABLY WRONG
UHS OF WEST

COMANCHE COUNTY HOSPITAL matched to COMANCHE COUNTY HOSPITAL

PROBABLY WRONG
METROPOLITAN HOSPITAL matched to METROPOLITAN HOSPITAL

PROBABLY WRONG
GREENE MEMORIAL HOSPITAL matched to GREENE MEMORIAL HOSPITAL

PROBABLY WRONG
GRADY MEMORIAL HOSPITAL matched to GRADY MEMORIAL HOSPITAL

PROBABLY WRONG
CENTRACARE HEALTH SYSTEM matched to CENTRACARE HEALTH SYSTEM

PROBABLY WRONG
HOSPITAL BUEN SAMARITANO matched to HOSPITAL BUEN SAMARITANO

PROBABLY WRONG
DUNCAN REGIONAL HOSPITAL matched to DUNCAN REGIONAL HOSPITAL

PROBABLY WRONG
TARZANA TREATMENT CENTER matched to TARZANA TREATMENT CENTER

PROBABLY WRONG
COLQUITT REGIONAL MEDICAL CENTER matched to COLQUITT REGIONAL MEDICAL CENTER

PROBABLY WRONG
KINGWOOD MEDICAL CENTER matched to KINGWOOD MEDICAL CENTER

PROBABLY WRONG
EATING RECOVERY CENTER matched to EATING RECOVERY CENTER

PROBABLY WRONG
MOUNT SINAI BETH ISRAEL matched to MOUNT SINAI BETH ISRAEL

PROBABLY WRONG
ST RITA'S MEDICAL CENTER matched to ST RITA'S MEDICAL CENTER

PROBABLY WRONG

PROBABLY WRONG
PROFESSIONAL RADIOLOGY PC matched to JOHN T MATHER MEMORIAL HOSPITAL OF PORT JEFFERSON NEW YORK INC

PROBABLY WRONG
SCRANTON CARDIOVASCULAR PHYSICIAN SERVICES LLC matched to REGIONAL HOSPITAL OF SCRANTON

PROBABLY WRONG
SKAGIT COUNTY ISLAND HOSPITAL - NAVY RESOURCE SHARING matched to ISLAND HOSPITAL

PROBABLY WRONG
HILLSBOROUGH RADIOLOGY CONSULTANTS, PL matched to SOUTH BAY HOSPITAL

PROBABLY WRONG
4499 ACUSHNET AVENUE OPERATING COMPANY matched to VIBRA HOSPITAL OF SOUTHWESTERN MASSACHUSETTS

PROBABLY WRONG
UNION EMERGENCY MEDICAL ASSOCIATES LLC matched to OVERLOOK MEDICAL CENTER

PROBABLY WRONG
DR. PERIKLIS PROUSSAEFS, A PROFESSIONAL DENTAL CORPORATION matched to GRIFFIN MEMORIAL HOSPITAL

PROBABLY WRONG
ELK REGIONAL HEALTH CENTER matched to PENN HIGHLANDS ELK

PROBABLY WRONG
PHYSICIANS HOSPITAL, LLC matched to MEDICAL BEHAVIORAL HOSPITAL - MISHAWAKA

PROBABLY WRONG
U S ANESTHESIA PARTNERS OF TEXAS, PA matched to CHI ST LUKES HEALTH - BRAZOSPORT

PROBABLY WRONG
ALBEMARL

PROBABLY WRONG
CAROLINAS PHYSICIANS NETWORK INC matched to CAROLINAS HEALTHCARE SYSTEM PINEVILLE

PROBABLY WRONG
IMAGING CONSULTANTS OF DENTON matched to ATRIUM MEDICAL CENTER AT CORINTH

PROBABLY WRONG
PLACER COUNTY ADULT SYSTEM OF CARE matched to TELECARE PLACER COUNTY PSYCHIATRIC HEALTH FACILITY

PROBABLY WRONG
FREEHOLD OBSERVATIONALIST PHYSICIANS, PC matched to CENTRASTATE MEDICAL CENTER

PROBABLY WRONG
BLUFF KNOLL EMERGENCY PHYSICIANS LLC matched to ST PETERSBURG GENERAL HOSPITAL

PROBABLY WRONG
SOUTH CAROLINS SCHOOL FOR THE DEAF AND BLIND matched to CHILDREN'S REHABILITATION CENTER

PROBABLY WRONG
COORDINATED PRIMARY CARE, INC. matched to HEALTH ALLIANCE HOSPITAL - LEOMINSTER CAMPUS

PROBABLY WRONG
TENNESSEE CANCER SPECIALISTS PLLC matched to TAKOMA ADVENTIST HOSPITAL

PROBABLY WRONG
SHERIDAN DROZDOW HEALTHCARE OF NEVADA PC matched to CENTENNIAL HILLS HOSPITAL MEDICAL CENTER

PROBABLY WRONG
BLUEGRASS CARDIOLOGY PSC matched to MARCUM AND WALLACE MEMORIAL HOSPITAL

PROBABLY WRONG
U

...

PROBABLY WRONG
BIO-MEDICAL APPLICATIONS OF LOUISIANA, LLC matched to SLIDELL -AMG SPECIALTY HOSPTIAL

PROBABLY WRONG
GREAT PLAINS OF SMITH COUNTY INC matched to SMITH COUNTY MEMORIAL HOSPITAL

PROBABLY WRONG
ROBERT C LEE DDS PHD PLLC matched to SMOKEY POINT BEHAIVORAL HOSPITAL

PROBABLY WRONG
BLUE RIDGE MEDICAL MANAGEMENT matched to SMYTH COUNTY COMMUNITY HOSPITAL

PROBABLY WRONG
SNOQUALMIE INDIAN TRIBE matched to SNOQUALMIE VALLEY HOSPITAL

PROBABLY WRONG
PALMGROOVE ENTERPRISES LLC matched to SO CRESCENT BEH HLTH SYS - ANCHOR HOSPITAL CAMPUS

PROBABLY WRONG
GOFORTH CHIROPRACTIC, LLC matched to SOCORRO GENERAL HOSPITAL

PROBABLY WRONG
REGENCY IHS OF BROWNSVILLE, LLC matched to SOLARA HOSPITAL HARLINGEN, BROWNSVILLE CAMPUS

PROBABLY WRONG
DAVID A. LEE, M.D. PLASTIC SURGERY ASSOCIATION matched to SOLARA HOSPITAL MCALLEN - EDINBURG

PROBABLY WRONG
PARK VIEW DENTAL ASSOC PC matched to SONOMA DEVELOPMENTAL CENTER

PROBABLY WRONG
SONOMA SPECIALTY HOSPITAL, LLC matched to SONOMA WEST MEDICAL 

PROBABLY WRONG
ST VINCENT HOSPITAL-HOSPITAL SISTERS-THIRD ORDER OF ST FRANCIS matched to ST VINCENT HSPTL

PROBABLY WRONG
CENTRAL ALABAMA MEDICAL ASSOCIATES, LLC matched to ST VINCENTS CHILTON

PROBABLY WRONG
STATE OF NEW YORK COMPTROLLERS OFFICE matched to ST. ALBANS COMMUNITY LIVING CENTER

PROBABLY WRONG
BETHESDA LONG TERM CARE, INC. matched to ST. ALEXIUS HOSPITAL - BROADWAY CAMPUS

PROBABLY WRONG
THE COMMUNITY WELLNESS PROJECT matched to ST. ALEXIUS HOSPITAL - JEFFERSON CAMPUS

PROBABLY WRONG
THE COMMUNITY WELLNESS PROJECT matched to ST. LOUIS PSYCHIATRIC REHABILITATION CENTER

PROBABLY WRONG
SSM HEALTHCARE OF OKLAHOMA, INC matched to ST. ANTHONY HOSPITAL

PROBABLY WRONG
HEALTHY MIND FOREVER matched to ST. BERNARD PARISH HOSPITAL

PROBABLY WRONG
PRESTIGE CARE, LLC matched to ST. CATHERINE MEMORIAL HOSPITAL

PROBABLY WRONG
WILLIAM BALOGH matched to ST. CLARE HOSPITAL

PROBABLY WRONG
ST. DAVID'S HEALTHCARE PARTNERSHIP, L.P., LLP matched to ST. DAVID'S MEDICAL CENTER

PROBABLY WRONG


PROBABLY WRONG
THR/STT SOUTHLAKE ASC, LLC matched to TEXAS HEALTH HARRIS METHODIST HOSPITAL SOUTHLAKE

PROBABLY WRONG
INNOVATION RECOVERY RESOURCE matched to TEXAS HEALTH HARRIS METHODIST HOSPITAL STEPHENVILLE

PROBABLY WRONG
FOCAL POINT OPTICAL INC matched to TEXAS HEALTH HEART & VASCULAR HOSPITAL  ARLINGTON

PROBABLY WRONG
E LINE ORTHODONTICS matched to TEXAS HEALTH HOSPITAL

PROBABLY WRONG
FOREST PARK MECIAL CENTER AT FORT WORTH, LLC matched to TEXAS HEALTH HOSPITAL CLEARFORK

PROBABLY WRONG
HISHAM BISMAR, MD matched to TEXAS HEALTH HUGULEY HOSPITAL

PROBABLY WRONG
RAY W NG DPM PA matched to TEXAS HEALTH PRESBYTERIAN HOSPITAL ALLEN

PROBABLY WRONG
ARVIN SHORT MD matched to TEXAS HEALTH PRESBYTERIAN HOSPITAL DENTON

PROBABLY WRONG
BRIDLEWOOD FAMILY HEALTHCARE, P.A. matched to TEXAS HEALTH PRESBYTERIAN HOSPITAL FLOWER MOUND

PROBABLY WRONG
PARKINSON PLACE SPEECHTHERAPY PLLC matched to TEXAS HEALTH SPRINGWOOD HOSPITAL HURST-EULESS-BEDFORD

PROBABLY WRONG
PATRICK H POWNELL MD P A matche

PROBABLY WRONG
VA HOSPITAL matched to VA MEDICAL CENTER - OMAHA

PROBABLY WRONG
PALO ALTO VA matched to VA MEDICAL CENTER - PALO ALTO DIVISION

PROBABLY WRONG
ALPA CARE CONCEPTS matched to VA MEDICAL CENTER - PHILADELPHIA

PROBABLY WRONG
VA NORTHERN CALIFORNIA matched to VA MEDICAL CENTER - SACRAMENTO

PROBABLY WRONG
KROSS FIRE COUNSELING & EQUINE ENTERPRISES matched to VA MEDICAL CENTER - SHERIDAN

PROBABLY WRONG
DEPARTMENT OF VETERANS AFFAIRS matched to VA MEDICAL CENTER - TOMAH

PROBABLY WRONG
TUSCALOOSA V.A. MEDICAL CENTER matched to VA MEDICAL CENTER - TUSCALOOSA

PROBABLY WRONG
VAPHS matched to VA MEDICAL CENTER - UNIVERSITY DRIVE CAMPUS

PROBABLY WRONG
LIFELINE CONNECTIONS matched to VA MEDICAL CENTER - VANCOUVER CAMPUS

PROBABLY WRONG
BOSTON VA HEALTHCARE SYSTEM matched to VA MEDICAL CENTER - WEST ROXBURY DIVISION

PROBABLY WRONG
SUSAN MCKENZIE PSYCHOTHERAPY PC matched to VA MEDICAL CENTER - WHITE RIVER JUNCTION

PROBABLY WRONG
VA MONTANA HEALTHCARE SYSTEM matched to VA MEDICAL


PROBABLY WRONG
ULTRA HEALTHCARE SERVICES INC matched to WINDMOOR HEALTHCARE OF CLEARWATER

PROBABLY WRONG
JVHD LLC matched to WINKLER COUNTY MEMORIAL HOSPITAL

PROBABLY WRONG
JULIO E. IGLESIAS,M.D. A PROFESSIONAL MEDICAL CORP matched to WINN PARISH MEDICAL CENTER

PROBABLY WRONG
WINNEBAGO INDIAN HEALTH SERVICE matched to WINNEBAGO HOSPITAL

PROBABLY WRONG
WINNESHIEK MEDICAL CENTER matched to WINNESHIEK COUNTY MEMORIAL HOSPITAL

PROBABLY WRONG
ROMERO REHABILITATION PHYSICAL THERAPY INC matched to WINSTON MEDICAL CETNER

PROBABLY WRONG
SWIFT HOME HEALTH CARE LLC matched to WINTER HAVEN WOMEN'S HOSPITAL

PROBABLY WRONG
COMPREHENSIVE DERMATOLOGY AND DERMATOLOGIC SURGERY, PLC matched to WINTER PARK MEMORIAL HOSPITAL

PROBABLY WRONG
BOOTH ENTERPRISE INC matched to WIREGRASS MEDICAL CENTER

PROBABLY WRONG
DECATUR HOSPITAL AUTHORITY matched to WISE REGIONAL HEALTH SYSTEM

PROBABLY WRONG
LAKEVIEW EAR NOSE & THROAT matched to WOMANS CHRISTIAN ASSOC HOSPITAL - WCA HOSP AT JONES MEMORIAL HEALTH C

In [63]:
all_match['Score']=scores

In [72]:
test = all_match.sort_values('Score', ascending=False).drop_duplicates('ID',keep='first')

In [84]:
unmatched_ids = list(test.sort_values('Score').head(800)['ID'])

In [92]:
unmatched_df = gov_hospitals[gov_hospitals['ID'].isin(unmatched_ids)][['NAME',
'ADDRESS',
'CITY',
'STATE',
'ZIP',
'TELEPHONE',
'STATUS',
'COUNTRY']]

In [101]:
unmatched_df = unmatched_df[unmatched_df.STATUS!='CLOSED']

In [102]:
unmatched_df[unmatched_df.COUNTRY!='USA']

,NAME,ADDRESS,CITY,STATE,ZIP,TELEPHONE,STATUS,COUNTRY
631,HOSPITAL INTERAMERICANO DE MEDICINA AVANZADA,100 LUIS MUNOZ MARIN AVENUE,CAGUAS,PR,726,(787) 653-3434,OPEN,PRI
666,PROFESSIONAL HOSPITAL,CARR. 199 KM 1.3,GUAYNABO,PR,970,(787) 708-6560,OPEN,PRI
784,CSF - UTUADO,C/ISAAC GONZALEZ MARTINEZ,ARENAS,PR,641,(787) 894-2875,OPEN,PRI
871,HOSPITAL PEREA,CALLE DR BASORA 15,MAYAGUEZ,PR,681,(787) 834-0101,OPEN,PRI
981,HEALTHSOUTH REHABILITATION HOSPITAL OF MANATI,"HOSPITALS DOCTOR CENTER, PR-2 KM 47.7",MANATI,PR,674,(787) 621-3800,OPEN,PRI
1364,HOSP PAVIA SANTURCE,1462 ASIA STREET,SANTURCE,PR,909,(787) 641-1616,OPEN,PRI
1367,SJ VETERANS ADMINISTRATION CARIBBEAN HEALTHCAR...,C/CASIA 10,MONACILLO URBANO,PR,921,(787) 641-7582,OPEN,PRI
2000,HOSPITAL SAN LUCAS DE GUAYAMA (CRISTO REDENTOR),AVENUE PEDRO ALBESUS,GUAYAMA PUEBLO,PR,785,(787) 840-8686,OPEN,PRI
2002,HOSP PSIQUIATRIA FORENSE DE PONCE,PR-14 KM 4.2,PONCE,PR,731,(787) 844-0101,OPEN,PRI
2003,MAYAGUEZ MEDICAL CENTER,410 HOSTOS AVENUE,MAYAGUEZ,PR,681,(787) 652-9200,OPEN,PRI


In [190]:
npi[npi['Prov_1st_Line_Bus_Prac_Loc_Addr']=='5350 LAMME RD']

,NPI,Taxonomy,Other,Prov_Org_Nm__Legal_Bus_Nm_,Prov_Other_Org_Nm,Prov_1st_Line_Bus_Mail_Addr,Prov_2nd_Line_Bus_Mail_Addr,Prov_Bus_Mail_Addr_City_Nm,Prov_Bus_Mail_Addr_State_Nm,Prov_Bus_Mail_Addr_Postal_Cd,...,Prov_Bus_Mail_Addr_Fax_Nbr,Prov_1st_Line_Bus_Prac_Loc_Addr,Prov_2nd_Line_Bus_Prac_Loc_Addr,Prov_Bus_Prac_Loc_Addr_City_Nm,Prov_Bus_Prac_Loc_Addr_State_Nm,Prov_Bus_Prac_Loc_Addr_Postal_Cd,Prov_Bus_Prac_Loc_Addr_Ctry_Cd,Prov_Bus_Prac_Loc_Addr_Phone_Nbr,Prov_Bus_Prac_Loc_Addr_Fax_Nbr,Last_Update_Date
895483,1679696314,207Q00000X,['207Q00000X'],FAMILY PRACTICE ASSOCIATES OF DAYTON LLC,NaN,PO BOX 78000,DEPT 781267,DETROIT,MI,452781267,...,9.37351e+09,5350 LAMME RD,NaN,MORAINE,OH,454393215,US,9374513123,9.37351e+09,01/19/2018
1044697,1780852228,2084P0804X,"['101YP1600X', '101YP2500X', '2084P0800X', '20...",ALLIANCE PHYSICIANS INC,KETTERING BEHAVIORAL MEDICINE CENTER,1 PRESTIGE PLACE,SUITE 550,MIAMISBURG,OH,453426115,...,9.37523e+09,5350 LAMME RD,NaN,MORAINE,OH,454393215,US,9375344632,9.37534e+09,11/16/2015


In [108]:
unmatched_df.to_csv('Unmatched.csv', index=False)

In [115]:
npi[npi['Taxonomy']=='320600000X']

,NPI,Taxonomy,Other,Prov_Org_Nm__Legal_Bus_Nm_,Prov_Other_Org_Nm,Prov_1st_Line_Bus_Mail_Addr,Prov_2nd_Line_Bus_Mail_Addr,Prov_Bus_Mail_Addr_City_Nm,Prov_Bus_Mail_Addr_State_Nm,Prov_Bus_Mail_Addr_Postal_Cd,...,Prov_Bus_Mail_Addr_Fax_Nbr,Prov_1st_Line_Bus_Prac_Loc_Addr,Prov_2nd_Line_Bus_Prac_Loc_Addr,Prov_Bus_Prac_Loc_Addr_City_Nm,Prov_Bus_Prac_Loc_Addr_State_Nm,Prov_Bus_Prac_Loc_Addr_Postal_Cd,Prov_Bus_Prac_Loc_Addr_Ctry_Cd,Prov_Bus_Prac_Loc_Addr_Phone_Nbr,Prov_Bus_Prac_Loc_Addr_Fax_Nbr,Last_Update_Date
597,1003021874,320600000X,"['315P00000X', '320600000X']","LIFETIME RESOURCES, INC.",ECHO FARMS GROUP HOME,1316 S. 16TH STREET,NaN,WILMINGTON,NC,284016422,...,9.10762e+09,220 DORCHESTER PL,NaN,WILMINGTON,NC,284127324,US,9107621189,9.10762e+09,07/07/2017
651,1003023789,320600000X,['320600000X'],ALTERNATIVE COMMUNITY LIVING INC,NEW PASSAGES,70 LAFAYETTE ST,NaN,PONTIAC,MI,483422033,...,2.48339e+09,36712 HATHAWAY ST,NaN,NEW BALTIMORE,MI,480473220,US,5867259594,5.86725e+09,04/25/2011
720,1003026642,320600000X,['320600000X'],"LOGAN COMMUNITY RESOURCES, INC",NaN,PO BOX 1049,NaN,SOUTH BEND,IN,466241049,...,5.74234e+09,2105 WABASH ST,NaN,SOUTH BEND,IN,466131138,US,5742894831,5.74234e+09,07/09/2007
1040,1003038779,320600000X,['320600000X'],BAPTIST CHILDRENS HOMES OF NC INC,THREE FORKS HOME,PO BOX 338,NaN,THOMASVILLE,NC,273610338,...,3.36474e+09,392 CAMP JOY RD,NaN,ZIONVILLE,NC,286989759,US,8282978001,NaN,07/10/2008
1438,1003055856,320600000X,['320600000X'],ZION DEVELOPMENT CORPORATION,NaN,7779 PINTAIL DR,NaN,FAYETTEVILLE,NC,283117405,...,9.1063e+09,1419 MILTON ST,NaN,SPRING LAKE,NC,283902511,US,9103390902,NaN,02/17/2009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1325751,1992929350,320600000X,['320600000X'],ADEC - HAWTHORNE,NaN,19670 SR 120,NaN,BRISTOL,IN,465070398,...,5.74849e+09,403 HAWTHORNE DR,NaN,GOSHEN,IN,465261211,US,5748487451,5.74849e+09,07/09/2007
1326086,1992944557,320600000X,['320600000X'],"SEQUEL OF NEW MEXICO, LLC",NaN,"5400 GIBSON BLVD., SE",SUITE 'A',ALBUQUERQUE,NM,87108,...,5.05769e+09,1801 RANDOLPH RD SE,NaN,ALBUQUERQUE,NM,871064230,US,5059246330,5.05769e+09,02/20/2018
1326968,1992977995,320600000X,['320600000X'],VOLUNTEERS OF AMERICA SOUTHEAST LOUISIANA,COMMINITY LIVING SERVICES,4152 CANAL ST,NaN,NEW ORLEANS,LA,701195941,...,5.04835e+09,4174 IBERVILLE ST,NaN,NEW ORLEANS,LA,701195139,US,5048353005,5.04835e+09,11/07/2018
1327010,1992979363,320600000X,['320600000X'],"LAURALEX, LLC",NaN,12516 RIVER RD,NaN,RICHMOND,VA,232386135,...,NaN,12516 RIVER RD,NaN,RICHMOND,VA,232386135,US,8043589300,NaN,06/25/2008


In [197]:
npi[(npi['Prov_Bus_Prac_Loc_Addr_City_Nm']=='MILLERSBURG')&(npi['Prov_Bus_Prac_Loc_Addr_State_Nm']=='OH')][0:50]

,NPI,Taxonomy,Other,Prov_Org_Nm__Legal_Bus_Nm_,Prov_Other_Org_Nm,Prov_1st_Line_Bus_Mail_Addr,Prov_2nd_Line_Bus_Mail_Addr,Prov_Bus_Mail_Addr_City_Nm,Prov_Bus_Mail_Addr_State_Nm,Prov_Bus_Mail_Addr_Postal_Cd,...,Prov_Bus_Mail_Addr_Fax_Nbr,Prov_1st_Line_Bus_Prac_Loc_Addr,Prov_2nd_Line_Bus_Prac_Loc_Addr,Prov_Bus_Prac_Loc_Addr_City_Nm,Prov_Bus_Prac_Loc_Addr_State_Nm,Prov_Bus_Prac_Loc_Addr_Postal_Cd,Prov_Bus_Prac_Loc_Addr_Ctry_Cd,Prov_Bus_Prac_Loc_Addr_Phone_Nbr,Prov_Bus_Prac_Loc_Addr_Fax_Nbr,Last_Update_Date
23092,1013398874,207Q00000X,['207Q00000X'],"HEALTH PROFESSIONALS OF HOLMEST COUNTY, INC.",POMERENE FAMILY CARE,1261 WOOSTER RD,SUITE 200,MILLERSBURG,OH,446541568,...,3.30763e+09,931 WOOSTER RD,NaN,MILLERSBURG,OH,446541536,US,3306748444,3.30675e+09,06/17/2015
70453,1053483438,291U00000X,['291U00000X'],HOLMES DIAGNOSTIC LABORATORY INC,NaN,1245 GLEN DRIVE,SUITE A,MILLERSBURG,OH,44654,...,8.66947e+09,1245 GLEN DRIVE,SUITE A,MILLERSBURG,OH,44654,US,3306746850,8.66947e+09,04/20/2008
88737,1063780294,111N00000X,['111N00000X'],CHAFFEE FAMILY CHIRO INC,NaN,242 E MILLTOWN RD,NaN,WOOSTER,OH,446911246,...,3.30346e+09,5336 C.R. 201,SUITE C,MILLERSBURG,OH,446549251,US,3308930444,3.30894e+09,05/01/2012
94633,1073077590,335E00000X,['335E00000X'],"SPICER ORTHOPEDICS, LLC",NaN,890 WOOSTER RD.,NaN,MILLERSBURG,OH,446541023,...,3.30473e+09,890 WOOSTER RD.,NaN,MILLERSBURG,OH,446541023,US,3305951010,3.30473e+09,01/29/2019
117773,1083897987,174400000X,['174400000X'],"HEALTH PROFESSIONALS OF HOLMES COUNTY, INC",POMERENE FAMILY CARE,1261 WOOSTER RD,SUITE 200,MILLERSBURG,OH,446541568,...,3.30763e+09,1261 WOOSTER RD,SUITE 200,MILLERSBURG,OH,446541568,US,3306742822,3.30763e+09,08/07/2012
161705,1124068994,314000000X,['314000000X'],"CASTLE NURSING HOMES, INC.",SCENIC POINTE NURSING & REHABILITATION,6967 DEER TRAIL AVE NE,NaN,CANTON,OH,447212069,...,NaN,8067 TOWNSHIP ROAD 334,NaN,MILLERSBURG,OH,446549171,US,3306740015,3307632230,05/07/2018
162913,1124128186,3336C0003X,['3336C0003X'],"RITZMAN PHARMACIES, INC",RITZMAN PHARMACY #214,2105 GLEN DR,P O BOX 109,MILLERSBURG,OH,446548905,...,3306744540,2105 GLEN DR,NaN,MILLERSBURG,OH,446548905,US,3306741891,3306744540,10/21/2013
165482,1124225644,251300000X,['251300000X'],WEST HOLMES LOCAL SCHOOLS,NaN,28 W JACKSON ST,NaN,MILLERSBURG,OH,446541302,...,3.30674e+09,28 W JACKSON ST,NaN,MILLERSBURG,OH,446541302,US,3306743546,3.30674e+09,07/08/2007
219533,1164691721,332B00000X,['332B00000X'],PERSONAL TOUCH UP,TOUCH OT TIME,PO BOX 108,NaN,MILLERSBURG,OH,44654,...,3.30675e+09,8612 SR 39 WEST,NaN,MILLERSBURG,OH,44654,US,3306747311,3.30675e+09,02/27/2008
227015,1174054704,174400000X,['174400000X'],"HEALTH PROFESSIONALS OF HOLMES COUNTY, INC",NaN,981 WOOSTER RD,NaN,MILLERSBURG,OH,446541536,...,3.30763e+09,1261 WOOSTER RD STE 230,NaN,MILLERSBURG,OH,446541570,US,3308931318,3.30763e+09,03/28/2019
